# **Tarea 3 - Word Embeddings 📚**

**Nombre: José Luis Cádiz Sejas**

**Fecha límite de entrega 📆:** 10 de mayo.

**Tiempo estimado de dedicación:**


**Instrucciones:**
- El ejercicio consiste en:
    - Responder preguntas relativas a los contenidos vistos en los vídeos y slides de las clases. 
    - Entrenar Word2Vec y Word Context Matrix sobre un pequeño corpus.
    - Evaluar los embeddings obtenidos en una tarea de clasificación.
- La tarea se realiza en grupos de **máximo** 2 personas. Puede ser invidivual pero no es recomendable.
- La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
- El formato de entrega es este mismo Jupyter Notebook.
- Al momento de la revisión tu código será ejecutado. Por favor verifica que tu entrega no tenga errores de compilación. 
- En el horario de auxiliar pueden realizar consultas acerca de la tarea a través del canal de Discord del curso. 


**Referencias**

Vídeos: 

- [Linear Models](https://youtu.be/zhBxDsNLZEA)
- [Neural Networks](https://youtu.be/oHZHA8h2xN0)
- [Word Embeddings](https://youtu.be/wtwUsJMC9CA)

## **Preguntas teóricas 📕 (2 puntos).** ##
Para estas preguntas no es necesario implementar código, pero pueden utilizar pseudo código.

### **Parte 1: Modelos Lineales (1 ptos)**

Suponga que tiene un dataset de 10.000 documentos etiquetados por 4 categorías: política, deporte, negocios y otros. 

**Pregunta 1**: Diseñe un modelo lineal capaz de clasificar un documento según estas categorías donde el output sea un vector con una distribución de probabilidad con la pertenencia a cada clase. 

Especifique: representación de los documentos de entrada, parámetros del modelo, transformaciones necesarias para obtener la probabilidad de cada etiqueta y función de pérdida escogida. **(0.5 puntos)**

**Respuesta**: A continuación se enumeran los pasos a seguir para diseñar un modelo ilneal que clasifique documentos.
1. **Representación**: Una representación básica podría ser One-Hot Vector, el cual consiste en representar cada oración del corpus como un vector sparse del tamaño del vacabulario, el cual tiene un 1 si la palabra aparece y 0 si no. Otra opción es Bag-of-Words (BOW), el cual también representa las oraciones como vectores sparse pero intenta capturar además, el orden de las palabras a traves de bigramas,trigramas etc. Esta representación vendrá dada por
el vector $\vec{x}$.

2. **Modelo**: Consideremos un modelo lineal de la forma $\vec{\hat{y}}=f(x)=\vec{x}\cdot W+\vec{b}$, donde $\vec{x}$ es el input. $W$, $\vec{b}$ son parámetros del modelo e $\vec{\hat{y}}$ es el Output de dimensión igual al número de etiquetas de la tarea de clasificación, en este caso dimensión 4.


3. **Transformaciones del Ouput para obtener probabilidad de etiqueta**: Para obtener una representación probabilistica del Output del modelo lineal, dicho Ouput es pasado por una función $\textit{Softmax}$ :

$\vec{\hat{y}}=\textit{Softmax}(\vec{x}\cdot W+\vec{b})$, donde la predicción estará dada por:  $Prediction=\hat{y}=argmax_{i}(\vec{\hat{y}}_{[i]})$, es decir, se asigna la etiqueta que tenga mayor probabilidad.


4. **Entrenamiento**: para mejorar la calidad de las predicciones se deben ajustar los parámetros del modelo de modo de maximizar la capacidad predictiva del modelo. Para esto es de vital importancia definir una
función de pérdida o también conocida como $\textit{Loss}$. Para una predicción dada, La función de $\textit{Loss}$, puede ser definida como
$\textit{Loss}$=$L(f(\vec{x},\vec{\Theta}),y_{true})$=$L(\hat{y},y_{true})$, donde $\vec{\Theta}$ representa los parámetros del modelo, $\hat{y}$ la predicción e $y_{true}$ la etiqueta correcta. Para considerar todas las predicciones que se harían sobre el conjunto de entrenamiento se define la función de $\textit{Loss}$ para todo el $\textit{Corpus}$: $\mathcal{L}(\vec{\Theta})=\frac{1}{N}\sum_{i=1}^{N}L(\hat{y_{i}},y_{true_{i}})$. Una función de pérdida adecuada para la tarea de clasificación es la conocida $\textit{Cross-Entropy-Loss}$. 


**Pregunta 2**: Explique cómo funciona el proceso de entrenamiento en este tipo de modelos y su evaluación. **(0.5 puntos)**

**Respuesta**: 

### **Parte 2: Redes Neuronales (1 ptos)** 

Supongamos que tenemos la siguiente red neuronal.

![image.png](https://drive.google.com/uc?export=view&id=1fFTjtMvH6MY8o42_vj010y8eTuCVb5a3)

**Pregunta 1**: En clases les explicaron como se puede representar una red neuronal de una y dos capas de manera matemática. Dada la red neuronal anterior, defina la salida $\vec{\hat{y}}$ en función del vector $\vec{x}$, pesos $W^i$, bias $b^i$ y funciones $g,f,h$. 

Adicionalmente liste y explicite las dimensiones de cada matriz y vector involucrado en la red neuronal. **(0.5 Puntos)**

**Respuesta**: 

Formula:
$\vec{\hat{y}} = NN_{MLP3}(\vec{x}) =$

Dimensiones: 

**Pregunta 2**: Explique qué es backpropagation. ¿Cuales serían los parámetros a evaluar en la red neuronal anterior durante backpropagation? **(0.25 puntos)**

**Respuesta**:

**Pregunta 3**: Explique los pasos de backpropagation. En la red neuronal anterior: Cuales son las derivadas que debemos calcular para poder obtener $\vec{\delta^l_{[j]}}$ en todas las capas? **(0.25 puntos)**

**Respuesta**:

## **Preguntas prácticas 💻 (4 puntos).** ##

### **Parte 3: Word Embeddings**

En la auxiliar 2 se nombraron dos formas de crear word vectors:

-  Distributional Vectors.
-  Distributed Vectors.

El objetivo de esta parte es comparar las dos embeddings obtenidos de estas dos estrategias en una tarea de clasificación.

In [3]:
import re  
import pandas as pd 
from time import time  
from collections import defaultdict 
import string 
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
import scipy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#### **Parte A (1 punto)** 

En esta parte debe crear una matriz palabra contexto, para esto, complete el siguiente template (para esta parte puede utilizar las librerías ```numpy``` y/o ```scipy```). Hint: revise como utilizar matrices sparse de ```scipy```

```python
class WordContextMatrix:

  def __init__(self, vocab_size, window_size, dataset, tokenizer):
    """
    Utilice el constructor para definir los parametros.
    """

    # se sugiere agregar un una estructura de datos para guardar las
    # palabras del vocab y para guardar el conteo de coocurrencia
    ...
    
  def add_word_to_vocab(self, word):
    """
    Utilice este método para agregar token
    a sus vocabulario
    """
    

    # Le puede ser útil considerar un token unk al vocab
    # para palabras fuera del vocab
    ...
  
  def build_matrix(self):
    """
    Utilice este método para crear la palabra contexto
    """
    ...

  def matrix2dict(self):
    """
    Utilice este método para convertir la matriz a un diccionario de embeddings, donde las llaves deben ser los token del vocabulario y los embeddings los valores obtenidos de la matriz. 
    """

    # se recomienda transformar la matrix a un diccionario de embedding.
    # por ejemplo {palabra1:vec1, palabra2:vec2, ...}
    ...

```

puede modificar los parámetros o métodos si lo considera necesario. Para probar la matrix puede utilizar el siguiente corpus.

```python
corpus = [
  "I like deep learning.",
  "I like NLP.",
  "I enjoy flying."
]
```

Obteniendo una matriz parecia a esta:

***Resultado esperado***: 

| counts   | I  | like | enjoy | deep | learning | NLP | flying | . |   
|----------|---:|-----:|------:|-----:|---------:|----:|-------:|--:|
| I        | 0  |  2   |  1    |    0 |  0       |   0 | 0      | 0|            
| like     |  2 |    0 |  0    |    1 |  0       |   1 | 0      | 0 | 
| enjoy    |  1 |    0 |  0    |    0 |  0       |   0 | 1      | 0 |
| deep     |  0 |    1 |  0    |    0 |  1       |   0 | 0      | 0 |  
| learning |  0 |    0 |  0    |    1 |  0       |   0 | 0      | 1 |          
| NLP      |  0 |    1 |  0    |    0 |  0       |   0 | 0      | 1 |
| flying   |  0 |    0 |  1    |    0 |  0       |   0 | 0      | 1 | 
| .        |  0 |    0 |  0    |    0 |  1       |   1 | 1      | 0 | 

``

**Respuesta:**

#### **Parte B (1.5 puntos)**

En esta parte es debe entrenar Word2Vec de gensim y construir la matriz palabra contexto utilizando el dataset de diálogos de los Simpson. 

Utilizando el dataset adjunto con la tarea:

In [ ]:
data_file = "dialogue-lines-of-the-simpsons.zip"
df = pd.read_csv(data_file)
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())
df = df.dropna().reset_index(drop=True) # Quitar filas vacias

2021-05-05 17:58:59,568 : INFO : NumExpr defaulting to 2 threads.


**Pregunta 1**: Ayudándose de los pasos vistos en la auxiliar, entrene los modelos Word2Vec. **(0.75 punto)** (Hint, le puede servir explorar un poco los datos)

**Respuesta**:

**Pregunta 2**: Cree una matriz palabra contexto usando el mismo dataset. Configure el largo del vocabulario a 1000 o 2000 tokens, puede agregar valores mayores pero tenga en cuenta que la construcción de la matriz puede tomar varios minutos. Puede que esto tarde un poco. **(0.75 punto)** 

**Respuesta:**

#### **Parte C (1.5 puntos): Aplicar embeddings para clasificar**

Ahora utilizaremos los embeddings que acabamos de calcular para clasificar palabras basadas en su polaridad (positivas o negativas). 

Para esto ocuparemos el lexicón AFINN incluido en la tarea, que incluye una lista de palabras y un 1 si su connotación es positiva y un -1 si es negativa.

In [ ]:
AFINN = 'AFINN_full.csv'
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)

Hint: Para w2v y la wcm son esperables KeyErrors debido a que no todas las palabras del corpus de los simpsons tendrán una representación en AFINN. Para el caso de la matriz palabra contexto se recomienda convertir su matrix a un diccionario. Pueden utilizar esta función auxiliar para filtrar las filas en el dataframe que no tienen embeddings (como w2v no tiene token UNK se deben ignorar).

In [ ]:
def try_apply(model,word):
    try:
        aux = model[word]
        return True
    except KeyError:
        #logger.error('Word {} not in dictionary'.format(word))
        return False

**Pregunta 1**: Transforme las palabras del corpus de AFINN a la representación en embedding que acabamos de calcular (con ambos modelos). 

Su dataframe final debe ser del estilo [embedding, sentimiento], donde los embeddings corresponden a $X$ y el sentimiento asociado con el embedding a $y$ (positivo/negativo, 1/-1). 

Para ambos modelos, separar train y test de acuerdo a la siguiente función. **(0.5 puntos)**

```python
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)
```


**Respuesta**:

**Pregunta 2**: Entrenar una regresión logística (vista en auxiliar) y reportar accuracy, precision, recall, f1 y confusion_matrix para ambos modelos. Por qué se obtienen estos resultados? Cómo los mejorarías? Como podrías mejorar los resultados de la matriz palabra contexto? es equivalente al modelo word2vec? **(1 punto)**

**Respuesta**:

# Bonus: +0.25 puntos en cualquier pregunta

**Pregunta 1**: Replicar la parte anterior utilizando embeddings pre-entrenados en un dataset más grande y obtener mejores resultados. Les puede servir [ésta](https://radimrehurek.com/gensim/downloader.html#module-gensim.downloader) documentacion de gensim **(0.25 puntos)**.

**Respuesta**: